In [1]:
import numpy as np
import sympy as sp

# ==============================================================
# 1. NumPy를 이용한 수치 해석적(eigendecomposition) 방법
# ==============================================================

# NumPy에서 사용할 2x2 행렬 A 정의 (dtype=float로 지정하여 부동소수점 계산)
A_np = np.array([[4, 2],
                 [1, 3]], dtype=float)

# 고유값(eigenvalues)과 고유벡터(eigenvectors) 계산
eigvals_np, eigvecs_np = np.linalg.eig(A_np)

print("===== NumPy를 이용한 결과 =====")
print("고유값:")
print(eigvals_np)
print("\n고유벡터 (각 열이 대응하는 벡터):")
print(eigvecs_np)

# ==============================================================
# 2. Sympy를 이용한 상징적(Symbolic) 방법
# ==============================================================

# Sympy에서 사용할 2x2 행렬 A 정의
A_sym = sp.Matrix([[7, -2],
                   [-2, 4]])

# 심볼 변수 lambda 생성
lambda_sym = sp.symbols('lambda')

# 특성다항식(Characteristic Polynomial) P(λ) = det(A - λI) 계산
char_poly = (A_sym - lambda_sym * sp.eye(2)).det()
print("\n===== Sympy를 이용한 결과 =====")
print("특성다항식 P(λ):")
sp.pprint(char_poly)  # 수학식 형태로 예쁘게 출력

# 특성다항식 P(λ)=0을 풀어 고유값들을 구함
eigen_vals = sp.solve(sp.Eq(char_poly, 0), lambda_sym)
print("\n고유값:")
print(eigen_vals)

# 각 고유값에 대해 (A - λI)의 nullspace를 구하여 고유벡터 계산
for val in eigen_vals:
    eig_vecs = (A_sym - val * sp.eye(2)).nullspace()
    print(f"\n고유값 λ = {val}에 대응하는 고유벡터:")
    for vec in eig_vecs:
        sp.pprint(vec)
    print("해 공간(고유공간)의 차원:", len(eig_vecs))


# ==============================================================
# 추가 설명: 고유벡터의 스칼라 배수
# ==============================================================
# 고유벡터는 스칼라 배수에 대해 본질적으로 같은 방향을 나타내므로,
# 예를 들어 λ=2일 때 고유벡터가 (1, -1)이라고 해도,
# (-1, 1)은 단순히 -1을 곱한 것과 같으므로 동일한 고유공간에 속합니다.
#
# 즉, (1, -1)과 (-1, 1)은 같은 고유벡터임을 의미합니다.


===== NumPy를 이용한 결과 =====
고유값:
[5. 2.]

고유벡터 (각 열이 대응하는 벡터):
[[ 0.89442719 -0.70710678]
 [ 0.4472136   0.70710678]]

===== Sympy를 이용한 결과 =====
특성다항식 P(λ):
 2            
λ  - 11⋅λ + 24

고유값:
[3, 8]

고유값 λ = 3에 대응하는 고유벡터:
⎡1/2⎤
⎢   ⎥
⎣ 1 ⎦
해 공간(고유공간)의 차원: 1

고유값 λ = 8에 대응하는 고유벡터:
⎡-2⎤
⎢  ⎥
⎣1 ⎦
해 공간(고유공간)의 차원: 1


In [9]:
import sympy as sp

# Sympy pretty-printing 활성화 (루트 기호 등 가독성 있게 표시)
sp.init_printing(use_unicode=True)

# ================================
# Step 0) 행렬 A 정의
# ================================
A = sp.Matrix([
    [3, 2, 2],
    [2, 3, 2],
    [2, 2, 3]
])
print("행렬 A:")
sp.pprint(A)

# 심볼 lambda(λ) 정의
lambda_sym = sp.Symbol('lambda', real=True)

# ================================
# Step 1) 특성다항식 (Characteristic polynomial)
# ================================
print("\n===== Step 1) Characteristic polynomial =====")
char_poly = (A - lambda_sym*sp.eye(3)).det()
print("P(λ) = ")
sp.pretty_print(char_poly)
print("\n인수분해 결과:")
sp.pretty_print(sp.factor(char_poly))

# ================================
# Step 2) 고유값 (Eigenvalues)
# ================================
print("\n===== Step 2) Eigenvalues =====")
eigenvals = sp.solve(sp.Eq(char_poly, 0), lambda_sym)
print("고유값 목록:")
for val in eigenvals:
    sp.pretty_print(val)

# ================================
# Step 3) 고유벡터 (Eigenvectors)
# ================================
print("\n===== Step 3) Eigenvectors =====")
eigen_dict = {}
for val in eigenvals:
    # (A - λI)x = 0 → nullspace() 사용
    eig_space = (A - val*sp.eye(3)).nullspace()
    eigen_dict[val] = eig_space
    
    print(f"\n고유값 λ = {val} 에 대한 고유벡터들:")
    for vec in eig_space:
        sp.pretty_print(vec)

# ================================
# Step 4) Gram–Schmidt를 이용한 직교(정규)화
# ================================
print("\n===== Step 4) Orthonormal eigenvectors (via Gram–Schmidt) =====")

def gram_schmidt(vecs):
    """주어진 벡터 목록 vecs를 Gram–Schmidt 알고리즘으로
       서로 직교하고 단위길이를 갖는 벡터들로 변환."""
    on_basis = []
    for v in vecs:
        w = sp.Matrix(v)  # 복사본
        for u in on_basis:
            # w에서 (w·u / u·u) * u를 빼서 직교 성분만 남김
            w = w - (w.dot(u) / u.dot(u)) * u
        # 길이가 너무 작지 않으면(=0이 아니면) 정규화 후 추가
        if w.norm() > 1e-14:
            w_unit = w / w.norm()
            on_basis.append(w_unit)
    return on_basis

# 최종적으로 저장할 직교(정규)화된 고유벡터들
orthonormal_vectors = []

# 서로 다른 고유값별로 얻은 벡터들을 각각 G–S 적용
for val in eigenvals:
    vecs = eigen_dict[val]
    gs_result = gram_schmidt(vecs)
    
    print(f"\n>> 고유값 λ={val}에 대한 고유벡터 Gram–Schmidt 결과:")
    for i, vec in enumerate(gs_result):
        print(f"  벡터 {i+1}:")
        sp.pretty_print(vec)
    # 전체 목록에 누적
    orthonormal_vectors.extend(gs_result)

# ================================
# 최종 요약: 모든 직교(정규) 고유벡터
# ================================
print("\n===== 최종 Orthonormal 고유벡터 집합 요약 =====")
for i, v in enumerate(orthonormal_vectors):
    print(f"벡터 {i+1}:")
    sp.pretty_print(v)


행렬 A:
⎡3  2  2⎤
⎢       ⎥
⎢2  3  2⎥
⎢       ⎥
⎣2  2  3⎦

===== Step 1) Characteristic polynomial =====
P(λ) = 
   3      2           
- λ  + 9⋅λ  - 15⋅λ + 7

인수분해 결과:
                2
-(λ - 7)⋅(λ - 1) 

===== Step 2) Eigenvalues =====
고유값 목록:
1
7

===== Step 3) Eigenvectors =====

고유값 λ = 1 에 대한 고유벡터들:
⎡-1⎤
⎢  ⎥
⎢1 ⎥
⎢  ⎥
⎣0 ⎦
⎡-1⎤
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎣1 ⎦

고유값 λ = 7 에 대한 고유벡터들:
⎡1⎤
⎢ ⎥
⎢1⎥
⎢ ⎥
⎣1⎦

===== Step 4) Orthonormal eigenvectors (via Gram–Schmidt) =====

>> 고유값 λ=1에 대한 고유벡터 Gram–Schmidt 결과:
  벡터 1:
⎡-√2 ⎤
⎢────⎥
⎢ 2  ⎥
⎢    ⎥
⎢ √2 ⎥
⎢ ── ⎥
⎢ 2  ⎥
⎢    ⎥
⎣ 0  ⎦
  벡터 2:
⎡-√6 ⎤
⎢────⎥
⎢ 6  ⎥
⎢    ⎥
⎢-√6 ⎥
⎢────⎥
⎢ 6  ⎥
⎢    ⎥
⎢ √6 ⎥
⎢ ── ⎥
⎣ 3  ⎦

>> 고유값 λ=7에 대한 고유벡터 Gram–Schmidt 결과:
  벡터 1:
⎡√3⎤
⎢──⎥
⎢3 ⎥
⎢  ⎥
⎢√3⎥
⎢──⎥
⎢3 ⎥
⎢  ⎥
⎢√3⎥
⎢──⎥
⎣3 ⎦

===== 최종 Orthonormal 고유벡터 집합 요약 =====
벡터 1:
⎡-√2 ⎤
⎢────⎥
⎢ 2  ⎥
⎢    ⎥
⎢ √2 ⎥
⎢ ── ⎥
⎢ 2  ⎥
⎢    ⎥
⎣ 0  ⎦
벡터 2:
⎡-√6 ⎤
⎢────⎥
⎢ 6  ⎥
⎢    ⎥
⎢-√6 ⎥
⎢────⎥
⎢ 6  ⎥
⎢    ⎥
⎢ √6 ⎥
⎢ ── ⎥
⎣ 3  ⎦
벡터 3:
⎡√3⎤
⎢──⎥
⎢3 ⎥
⎢  ⎥
⎢√3⎥
⎢──⎥
⎢3 ⎥
⎢  ⎥
⎢√3⎥
